## Preprocessing the Dataset

In [34]:
import os
import pandas as pd

# read the file in the folder
path = os.listdir('./redbus_details/')
file_path = [ os.path.join("./redbus_details/", file) for file in path]

df = [ pd.read_csv(file) for file in file_path]

# concatenate the all dataframe 
df_comb = pd.concat(df, ignore_index=True)

# Remove the INR in the price column
df_comb['Price'] = df_comb['Price'].str.replace('INR', '')

# Extract the numeric value in the seat_availability column
df_comb['Seat_Availability'] = df_comb['Seat_Availability'].str.extract(r"(\d+)")


# Add the id column to the dataset at first
df_comb.insert(loc=0, column='id', value=range(1, len(df_comb)+1))

df_comb.to_csv("redbus_routes.csv", index=False)


## Store the data into database

In [1]:
from sqlite3 import connect

conn = connect('redbus_db.db')
curr = conn.cursor()

In [9]:
import pandas as pd
df = pd.read_csv('./redbus_routes.csv')




In [67]:
# create a table
table = """
    CREATE TABLE redbus_details (
        id INT PRIMARY KEY,
        bus_name VARCHAR(255),
        route_name VARCHAR(255),
        bus_type VARCHAR(255),
        departing_time VARCHAR(150),
        duration VARCHAR(100),
        reaching_time VARCHAR(150),
        star_rating  DECIMAL(3,2),
        price DECIMAL(10,2),
        seat_availability INT,
        route_link VARCHAR(255)  
    );
"""

In [80]:
curr.execute(table)

In [100]:
# insert the data into the table
sql_query = """
    INSERT INTO redbus_details (
        id, bus_name, route_name, bus_type, departing_time,
        duration, reaching_time, star_rating, price, seat_availability, route_link
    ) VALUES 
    (:id, :bus_name, :route_name, :bus_type, :departing_time,
    :duration, :reaching_time, :star_rating, :price, :seat_availability, :route_link);
"""


# for index,row in df.iterrows():
#     curr.execute(
#        sql_query,
#         (
#             row['id'],
#             row['Bus_Name'],
#             row['Route_Name'],
#             row['Bus_Type'],
#             row['Departing_Time'],
#             row['Duration'],
#             row['Reaching_Time'],
#             row['Star_Rating'],
#             row['Price'],
#             row['Seat_Availability'],
#             row['Route_Link']
#         )
#     )
#     conn.commit()
    
for index, row in df.iterrows():
    params = {
        'id': row['id'],
        'bus_name': row['Bus_Name'],
        'route_name': row['Route_Name'],
        'bus_type': row['Bus_Type'],
        'departing_time': row['Departing_Time'],
        'duration': row['Duration'],
        'reaching_time': row['Reaching_Time'],
        'star_rating': row['Star_Rating'],
        'price': row['Price'],
        'seat_availability': row['Seat_Availability'],
        'route_link': row['Route_Link']
    }
    curr.execute(sql_query, params)
    conn.commit()
    

In [97]:
# curr.execute("delete from  redbus_details;")

In [ ]:
data = curr.execute("select * from redbus_details order by price asc limit 10 offset 500 ;").fetchall()
for i in data:
    print(i)
    